In [57]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import pandas as pd
import numpy as np

In [2]:
country_gender_data = pd.read_csv("LifeExp_vs_Year.csv").drop(['Unnamed: 0', 'Country Name'],axis = 1)
bmi = pd.read_csv('females_males_just_BMI.csv').drop('Unnamed: 0',axis = 1)


In [3]:
bmi_male = bmi[['country','year', 'BMI_male']].rename(columns = {'BMI_male': 'BMI'})
bmi_male['Gender'] = 'Population, male'
bmi_male
bmi_female = bmi[['country','year', 'BMI_females']].rename(columns = {'BMI_females': 'BMI'})
bmi_female['Gender'] = 'Population, female'
bmi_female
final_bmi = pd.concat([bmi_male, bmi_female], ignore_index=True)
final_bmi = final_bmi.groupby(['country','Gender'])\
[['BMI']].mean().reset_index()

In [4]:
country_gender_data = country_gender_data.rename(columns = {'Indicator Name': 'Gender'})

In [5]:
country_gender_data = country_gender_data.groupby(['country','Gender'])\
[['LifeExpectancy']].mean().reset_index()

In [6]:
temp = final_bmi.merge(country_gender_data, on = ['country', 'Gender'])

In [7]:
temp

country              Gender        BMI  LifeExpectancy
0    Afghanistan  Population, female  20.668966       51.168852
1    Afghanistan    Population, male  21.013793       48.931148
2        Albania  Population, female  25.189655       74.334426
3        Albania    Population, male  25.510345       69.998361
4        Algeria  Population, female  25.003448       65.501639
..           ...                 ...        ...             ...
335      Vietnam    Population, male  19.744828       64.721311
336       Zambia  Population, female  22.058621       53.418033
337       Zambia    Population, male  20.075862       49.803279
338     Zimbabwe  Population, female  24.868966       56.801639
339     Zimbabwe    Population, male  21.665517       52.832787

[340 rows x 4 columns]

In [8]:
# prediction for life expectancy given its gender, its country, 
# its bmi, its income, its happnies score
# the above factors include economic, physical and mental conditions, which should be comprehensive

In [97]:
enc = OneHotEncoder(handle_unknown = "ignore")
X = np.array(temp.get(["country", "Gender"]))
enc.fit(X)

OneHotEncoder(handle_unknown='ignore')

In [98]:
df_X = pd.DataFrame(enc.transform(X).toarray(), columns = enc.get_feature_names())
df_Y = pd.DataFrame(temp["LifeExpectancy"])

/Users/saathvikdirisala/Library/Python/3.8/lib/python/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [99]:
BMI = np.array(temp["BMI"]).reshape(len(temp), 1)
scaler = StandardScaler().fit(BMI)
BMI_scaled = scaler.transform(BMI)
df_X["BMI"] = BMI_scaled

In [100]:
#Input:
country = "Afghanistan"
gender = "Male"
bmi = 24

#Computation:
reg = LinearRegression().fit(np.array(df_X), np.array(df_Y))
input0 = enc.transform([[country, "Population, " + gender.lower()]]).toarray()
scaled_BMI_input = scaler.transform([[bmi]])
input0 = np.append(input0, scaled_BMI_input[0]).reshape(1, len(df_X.columns))
pred = reg.predict(input0)[0][0]
print("Life Expectancy: " + str(pred) + " years")

Life Expectancy: 47.4375 years
